<a href="https://colab.research.google.com/github/loicpages/qmII_group_rep/blob/main/No_missing_values_data_with_geographical_regions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Data cleaning notebook - use to clean/ merge/ rearrange data sets***

*Libraries used:*

In [21]:
import pylab
import pandas as pd
import numpy as np
from google.colab import files
import io
import matplotlib.pyplot as plt
from functools import reduce
%matplotlib inline
plt.style.use('ggplot')

*Importing, Encording and Indexing data:*

In [22]:
data_path = '/content/complete_data_set.csv'
general_df = pd.read_csv(data_path, encoding ='latin-1')

In [23]:
#Creating a food_availibility dataframe#
intermediate_food_availibility_df = general_df[general_df.Commodity == "Total Grains/Cereals and Root Crops (R&T)"]
food_availibility_df = intermediate_food_availibility_df[intermediate_food_availibility_df.Item == 'Food Availability per capita']
food_availibility_df = food_availibility_df.reset_index(drop = True)
food_availibility_df.rename(columns = {'Amount': 'Food Availibility per capita (kg/cap/year)'}, inplace = True)
food_availibility_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating a food_production dataframe#
intermediate_food_production_df = general_df[general_df.Commodity == "Total Grains/Cereals and Root Crops (R&T)"]
food_production_df = intermediate_food_production_df[intermediate_food_production_df.Item == 'Production Quantity']
food_production_df = food_production_df.reset_index(drop = True)
food_production_df.rename(columns = {'Amount': 'Production Quantity (Grains & Ce.+ Root Crops)(1000MT)'}, inplace = True)
food_production_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating an import_grains_df#
intermediate_import_grains_df = general_df[general_df.Commodity == "Total Grains/Cereals"]
import_grains_df = intermediate_import_grains_df[intermediate_import_grains_df.Item == 'Import Quantity']
import_grains_df = import_grains_df.reset_index(drop = True)
import_grains_df.rename(columns = {'Amount': 'Import Quantity (Grains & Ce.)(1000MT)'}, inplace = True)
import_grains_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating an export_grains_df#
intermediate_export_grains_df = general_df[general_df.Commodity == "Total Grains/Cereals"]
export_grains_df = intermediate_export_grains_df[intermediate_export_grains_df.Item == 'Export Quantity']
export_grains_df = export_grains_df.reset_index(drop = True)
export_grains_df.rename(columns = {'Amount': 'Export Quantity (Grains & Ce.)(1000MT)'}, inplace = True)
export_grains_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating an import_root_df#
intermediate_import_root_df = general_df[general_df.Commodity == "Root Crops (R&T)"]
import_root_df = intermediate_import_root_df[intermediate_import_root_df.Item == 'Import Quantity']
import_root_df = import_root_df.reset_index(drop = True)
import_root_df.rename(columns = {'Amount': 'Import Quantity (Root Crops)(1000MT)'}, inplace = True)
import_root_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating an export_root_df#
intermediate_export_root_df = general_df[general_df.Commodity == "Root Crops (R&T)"]
export_root_df = intermediate_export_root_df[intermediate_export_root_df.Item == 'Export Quantity']
export_root_df = export_root_df.reset_index(drop = True)
export_root_df.rename(columns = {'Amount': 'Export Quantity (Root Crops)(1000MT)'}, inplace = True)
export_root_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)
#Creating a population_df#
population_df = general_df[general_df.Commodity == "Population"]
population_df = population_df.reset_index(drop = True)
population_df.rename(columns = {'Amount': 'Population (Million)'}, inplace = True)
population_df.drop(['Commodity', 'Item', 'Unit'], axis=1, inplace = True)

In [24]:
#merge data frames and fill gaps#
data_frames = [food_availibility_df, food_production_df, import_grains_df, export_grains_df, import_root_df, export_root_df, population_df]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Country', 'Year'],
                                            how='outer'), data_frames).fillna(pd.NaT)    

df_merged.sort_values(by=['Country', 'Year'], inplace = True)
df_merged.reset_index(drop = True)

,Country,Year,Food Availibility per capita (kg/cap/year),Production Quantity (Grains & Ce.+ Root Crops)(1000MT),Import Quantity (Grains & Ce.)(1000MT),Export Quantity (Grains & Ce.)(1000MT),Import Quantity (Root Crops)(1000MT),Export Quantity (Root Crops)(1000MT),Population (Million)
0,Afghanistan,1980,NaT,3933.8,17.1,0,0,0,14.186
1,Afghanistan,1981,NaT,3763.13,96.5,0,0,0,13.984
2,Afghanistan,1982,NaT,3602.79,74.9,0,0,0,13.673
3,Afghanistan,1983,NaT,3474.45,79.503,0,0,0,13.3
4,Afghanistan,1984,NaT,3306.78,21.15,0,0,0,12.932
...,...,...,...,...,...,...,...,...,...
2579,Zimbabwe,2009,141.317,1677.73,576.3,0,2.16398,0.0144,12.474
2580,Zimbabwe,2010,136.991,1723.57,368.6,0,2.16398,0.0144,12.571
2581,Zimbabwe,2011,139.933,1765.47,640,0,2.16398,0.0144,12.754
2582,Zimbabwe,2012,142.319,1255.9,620,0,2.16398,0.0144,13.014


In [25]:
#Add additional columns that are required for analysis#
df_merged['Imports (all)(1000MT)'] = df_merged['Import Quantity (Grains & Ce.)(1000MT)'] + df_merged['Import Quantity (Root Crops)(1000MT)']
df_merged['Exports (all)(1000MT)'] = df_merged['Export Quantity (Grains & Ce.)(1000MT)'] + df_merged['Export Quantity (Root Crops)(1000MT)']
df_merged['Production Quantity per capita (kg/cap/year)'] = df_merged['Production Quantity (Grains & Ce.+ Root Crops)(1000MT)'] / df_merged['Population (Million)']
df_merged['Imports (all) per capita (kg/cap/year)'] = df_merged['Imports (all)(1000MT)'] / df_merged['Population (Million)']
df_merged['Exports (all) per capita (kg/cap/year)'] = df_merged['Exports (all)(1000MT)'] / df_merged['Population (Million)']
df_merged['Production + net ex-imports (kg/cap/year)'] = (df_merged['Imports (all) per capita (kg/cap/year)'] + df_merged['Production Quantity per capita (kg/cap/year)']) - df_merged['Exports (all) per capita (kg/cap/year)']

#save as csv file#
df_merged.to_csv(r'df_final.csv', index = False)

#Save as csv without missing values#
df_no_missing_values = df_merged.dropna()
df_no_missing_values.to_csv(r'df_final(no_missing values).csv', index = False)


**Countries:**  Algeria, Angola, Armenia, Azerbaijan, Bangladesh, Benin, Bolivia, Burkina Faso, Burundi, CAR, Cambodia, Cameroon, Cape Verde, Chad, Colombia, Congo, C√¥te d'Ivoire, Egypt, Eritrea, Ethiopia, Gambia, Georgia, Ghana, Guinea, Guinea-Bissau, India, Indonesia, Kenya, Kyrgyzstan, Laos, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Moldova, Mongolia, Morocco, Mozambique, Namibia, Nepal, Niger, Nigeria, North Korea, Pakistan, Philippines, Rwanda, Senegal, Sierra Leone, Sri Lanka, Sudan, Swaziland, Tajikistan, Tanzania, Togo, Tunisia, Turkmenistan, Uganda, Uzbekistan, Vietnam, Yemen, Zambia, Zimbabwe

**Regions:**
North Africa, Central Africa, West Africa, East Africa, Southern Africa

Caucasus and Middle East 

South Asia, South East Asia, Central and East Asia

South America

Europe

In [26]:
data_path = '/content/df_final(no_missing values).csv'
df_no_missing_values = pd.read_csv(data_path, encoding ='latin-1')
conditions = [
#North Africa
    (df_no_missing_values['Country'] == 'Algeria') |
    (df_no_missing_values['Country'] == 'Mauritania') |
    (df_no_missing_values['Country'] == 'Morocco') |
    (df_no_missing_values['Country'] == 'Tunisia'),
#Central Africa
    (df_no_missing_values['Country'] == 'Angola') | 
    (df_no_missing_values['Country'] == 'Cameroon') | 
    (df_no_missing_values['Country'] == 'Chad') | 
    (df_no_missing_values['Country'] == 'Congo') |
    (df_no_missing_values['Country'] == 'Rwanda') |
    (df_no_missing_values['Country'] == 'Sudan'),
#Caucasus and Middle East
    (df_no_missing_values['Country'] == 'Armenia') | 
    (df_no_missing_values['Country'] == 'Azerbaijan') | 
    (df_no_missing_values['Country'] == 'Egypt') | 
    (df_no_missing_values['Country'] == 'Georgia') |
    (df_no_missing_values['Country'] == 'Tajikistan') |
    (df_no_missing_values['Country'] == 'Yemen'),
#South Asia
    (df_no_missing_values['Country'] == 'Bangladesh') |
    (df_no_missing_values['Country'] == 'India') |
    (df_no_missing_values['Country'] == 'Nepal') |
    (df_no_missing_values['Country'] == 'Pakistan') |
    (df_no_missing_values['Country'] == 'Sri Lanka'),
#West Africa
    (df_no_missing_values['Country'] == 'Benin') | 
    (df_no_missing_values['Country'] == 'Burkina Faso') | 
    (df_no_missing_values['Country'] == 'Cape Verde') | 
    (df_no_missing_values['Country'] == "C√¥te d'Ivoire") | 
    (df_no_missing_values['Country'] == 'Gambia') |
    (df_no_missing_values['Country'] == 'Ghana') |
    (df_no_missing_values['Country'] == 'Guinea') |
    (df_no_missing_values['Country'] == 'Guinea-Bissau') |
    (df_no_missing_values['Country'] == 'Liberia') |
    (df_no_missing_values['Country'] == 'Mali') |
    (df_no_missing_values['Country'] == 'Niger') |
    (df_no_missing_values['Country'] == 'Nigeria') |
    (df_no_missing_values['Country'] == 'Senegal') |
    (df_no_missing_values['Country'] == 'Sierra Leone') |
    (df_no_missing_values['Country'] == 'Togo'),
#South America
    (df_no_missing_values['Country'] == 'Bolivia') | 
    (df_no_missing_values['Country'] == 'Colombia'),
#East Africa
    (df_no_missing_values['Country'] == 'Burundi') | 
    (df_no_missing_values['Country'] == 'Eritrea') | 
    (df_no_missing_values['Country'] == 'Ethiopia') |
    (df_no_missing_values['Country'] == 'Kenya') |
    (df_no_missing_values['Country'] == 'Tanzania') |
    (df_no_missing_values['Country'] == 'Uganda'),
#South East Asia
    (df_no_missing_values['Country'] == 'CAR') | 
    (df_no_missing_values['Country'] == 'Cambodia') |
    (df_no_missing_values['Country'] == 'Indonesia') |
    (df_no_missing_values['Country'] == 'Laos') |
    (df_no_missing_values['Country'] == 'Philippines') |
    (df_no_missing_values['Country'] == 'Vietnam'),
#Central and East Asia
    (df_no_missing_values['Country'] == 'Kyrgyzstan') |
    (df_no_missing_values['Country'] == 'Mongolia') |
    (df_no_missing_values['Country'] == 'North Korea') |
    (df_no_missing_values['Country'] == 'Turkmenistan') |
    (df_no_missing_values['Country'] == 'Uzbekistan'),
#Southern Africa
    (df_no_missing_values['Country'] == 'Lesotho') |
    (df_no_missing_values['Country'] == 'Madagascar') |
    (df_no_missing_values['Country'] == 'Malawi') |
    (df_no_missing_values['Country'] == 'Mozambique') |
    (df_no_missing_values['Country'] == 'Namibia') |
    (df_no_missing_values['Country'] == 'Swaziland') |
    (df_no_missing_values['Country'] == 'Zambia') |
    (df_no_missing_values['Country'] == 'Zimbabwe'),
#Europe
    (df_no_missing_values['Country'] == 'Moldova')]

values = ['North Africa', 'Central Africa', 'Caucasus and Middle East', 'South Asia', 'West Africa', 'South America', 'East Africa', 'South East Asia', 'Central and East Asia', 'Southern Africa', 'Europe']
df_no_missing_values['Region'] = np.select(conditions, values)

df_no_missing_values.to_csv(r'df_final(no_missing values,regions).csv', index = False)